In [2]:
import pandas as pd
import numpy as np

import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [3]:
# Set the OMP_NUM_THREADS environment variable to 1 to avoid memory leak
os.environ['OMP_NUM_THREADS'] = '1'

#### Without Context

In [21]:
df1 = pd.read_csv('data/two_treatment/dt0_without_context.csv' ,index_col=0)
df1.head()

,t,patient_id,treatment,outcome
0,0,0,0,0.125730
1,1,0,0,-0.132105
2,2,0,0,0.640423
3,3,0,0,0.104900
4,4,0,0,-0.535669


#### creating random nan values

In [6]:
df = df1.copy()
column = 'outcome' ## clumn tht ill have missing value
nan_fraction = 0.1  
total_rows = len(df)
nan_count = int(total_rows * nan_fraction)

# seed for reproducibility
np.random.seed(0)
nan_indices = np.random.choice(df.index, nan_count, replace=False)

# Set those randomly selected positions to NaN in the specified column
df.loc[nan_indices, column] = np.nan


#### Separating the dataframe to calculate missing data
In a bayesian adaptive N_of_1 trials, we are going to calculate the missing data as we prepare to update the posterior joint distribution. Thus, the first step is to sepaate the dataframe when the first missing value appears.

In [7]:
## seperating the dataframe at the point of first missing value
nan_t = df[df.isna().any(axis=1)]['t']
nan_t = nan_t.sort_values(ascending=True).unique() ## shorting the value in ascending order to make sure we dot the first time cycle
ts = nan_t[0]
dt = df[df['t'] <= ts].copy()
dt.head()
     

,c,t,patient_id,treatment,outcome
0,0.125730,0,0,0,-0.132105
30,0.345584,0,1,0,NaN
60,0.189053,0,2,1,-0.522748
90,2.040919,0,3,1,-2.555665
120,-0.651791,0,4,0,-0.174717


Now, we have data set where we encountered the first missig value in the experiments. lets fill the missing value with various method.
#### Context Clustering filling method
For the context Clustering filling method, we are followin similar algorithm as Clustering to select suitable filling value. The context of the each patient and the treatment altogether is a vector $x_t$(i.e for patient id 1 the vector is $[c_1, c_2, ...., c_n, treatment]_{p_1}$). First, we cluster all $x_t$ in $N$ clusters with corresponding outcomes ($r$) , and claculate the each cluster's mean and centroids. Now, similar to KNN imputation method, we select $m$ nearest clusters' means. Finally, a weighted average of the clusters' means with distnce is taken to fill the missing value. the weighted average is as follows,

\begin{equation}
        g(x_t) = \frac{\sum_{j=1}^{m}\frac{\bar{r_j}}{d_j}}{\sum_{j=1}^{m}\frac{1}{d_j}}
\end{equation}
where $d_j$ represents the distance between $x_t$ and the centroid of $j^{th}$ cluster and $\bar{r_j}$ (j th cluster mean) given by,
\begin{equation}
\tag{2}
        \bar{r_j} = \frac{\sum_{\tau = 1}^{n_j} r_\tau}{n_j}
\end{equation}

If no context is available, then we are using the treatment as a singular context.

In [8]:
def get_distance(context_vectors, miss_vec):
    
    distances = []
    for vector in context_vectors:
        d = np.linalg.norm(vector - miss_vec)
        distances.append(d) 
    return(distances)

In [9]:
def context_fill(dt, context_cols = list, N = 2, m = 1): 
    # dt is the data Frame
    # context_cols will be list of columns name which are considered as context 
    # N represents the number of clusters for KMean
    
    dt_fill = dt.copy()
    miss_dt = dt_fill[dt_fill['outcome'].isna()]
    without_miss_dt = dt_fill[dt_fill['outcome'].isna() == False].copy()
    context_dt = without_miss_dt[context_cols]
    
    ## apply KMeans clustering to context_dt
    
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(context_dt)

    kmeans = KMeans(n_clusters= N, random_state=0)  # Choose the number of clusters (2 in this case)
    kmeans.fit(scaled_data)

    without_miss_dt['cluster'] = kmeans.labels_  # Add cluster labels to the dataframe
    centroids = kmeans.cluster_centers_  # Get cluster centroids
    
    
    
    
    for row in miss_dt.itertuples():
        miss_vec = np.array([getattr(row, col) for col in context_cols])
        
        ## calculate the distance from the clustres cenroids
        dis = get_distance(centroids, miss_vec)
        dis_dt = pd.Series(dis)
                
        sorted_dis_dt = dis_dt.sort_values()
        selected_cluster = sorted_dis_dt.index[:m].values ## to get m nearest clusters
        
        ## calculate the missing value with above mentioned equation (1)
        clusters_mean = without_miss_dt.groupby('cluster')['outcome'].mean()
        inverse_dis = 1 / dis_dt.loc[selected_cluster]
        numerator = (clusters_mean.loc[selected_cluster] / dis_dt.loc[selected_cluster]).mean()
        denominator = inverse_dis.mean()
        
        fill_value = numerator/denominator 
        
        ## fill the missing value in appropiate place 
        dt_fill.loc[row.Index,'outcome'] = fill_value
    
    return dt_fill

**while t = 0**

In [11]:
N = 2
m = 1
context_cols = ['treatment']

print(dt.isna().value_counts())
filled_dt = context_fill(dt, context_cols, N, m)
print(filled_dt.isna().value_counts())


c      t      patient_id  treatment  outcome
False  False  False       False      False      86
                                     True       14
Name: count, dtype: int64
c      t      patient_id  treatment  outcome
False  False  False       False      False      100
Name: count, dtype: int64


c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


**when t =! 0**

In [12]:
next_cycle = df[(df['t']> nan_t[0]) & (df['t'] <= nan_t[1])].copy()
next_dt = pd.concat([filled_dt, next_cycle], axis= 0, ignore_index=False) # add the next cycle to the data
next_dt = next_dt.sort_index()
next_dt

,c,t,patient_id,treatment,outcome
0,0.125730,0,0,0,-0.132105
1,0.640423,1,0,0,0.104900
30,0.345584,0,1,0,-0.200820
31,0.330437,1,1,0,-1.303157
60,0.189053,0,2,1,-0.522748
...,...,...,...,...,...
2911,-0.425905,1,97,0,-0.985005
2940,-1.087591,0,98,0,-1.244621
2941,0.560527,1,98,0,NaN
2970,0.082494,0,99,1,-0.464418


In [13]:
print(next_dt.isna().value_counts())
next_filled_dt = context_fill(next_dt, context_cols, N, m)
print(next_filled_dt.isna().value_counts())

c      t      patient_id  treatment  outcome
False  False  False       False      False      188
                                     True        12
Name: count, dtype: int64
c      t      patient_id  treatment  outcome
False  False  False       False      False      200
Name: count, dtype: int64


c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


#### With Context data

In [15]:
df2 = pd.read_csv('data/two_treatment/dt0_with_context.csv' ,index_col=0)
df2.head()

,c,t,patient_id,treatment,outcome
0,0.125730,0,0,0,-0.132105
1,0.640423,1,0,0,0.104900
2,-0.535669,2,0,0,0.361595
3,1.304000,3,0,0,0.947081
4,-0.703735,4,0,0,-1.265421


#### creating random nan values

In [16]:
df = df2.copy()
column = 'outcome'
nan_fraction = 0.1  

total_rows = len(df)
nan_count = int(total_rows * nan_fraction)

# seed for reproducibility
np.random.seed(0)
nan_indices = np.random.choice(df.index, nan_count, replace=False)

# Set those randomly selected positions to NaN in the specified column
df.loc[nan_indices, column] = np.nan


#### Separating the dataframe to calculate missing data
In a bayesian aaptive N_of_1 trials, we are going to calculate the missing data as we prepare to update the posterior joint distribution. Thus, the first step is to sepaate the dataframe when the first missing value appears.

In [17]:
## seperating the dataframe at the point of first missing value
nan_t = df[df.isna().any(axis=1)]['t']
nan_t = nan_t.sort_values(ascending=True).unique() ## shorting the value in ascending order to make sure we dot the first time cycle
ts = nan_t[0]
dt = df[df['t'] <= ts].copy()
dt.head()

,c,t,patient_id,treatment,outcome
0,0.125730,0,0,0,-0.132105
30,0.345584,0,1,0,NaN
60,0.189053,0,2,1,-0.522748
90,2.040919,0,3,1,-2.555665
120,-0.651791,0,4,0,-0.174717


**while t = 0**

In [18]:
N = 2
m = 1
context_cols = ['c', 'treatment']

print(dt.isna().value_counts())
filled_dt = context_fill(dt, context_cols, N, m)
print(filled_dt.isna().value_counts())

c      t      patient_id  treatment  outcome
False  False  False       False      False      86
                                     True       14
Name: count, dtype: int64
c      t      patient_id  treatment  outcome
False  False  False       False      False      100
Name: count, dtype: int64


c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


**while t != 0**

In [19]:
next_cycle = df[(df['t']> nan_t[0]) & (df['t'] <= nan_t[1])].copy()
next_dt = pd.concat([filled_dt, next_cycle], axis= 0, ignore_index=False) # add the next cycle to the data
next_dt = next_dt.sort_index()
next_dt

,c,t,patient_id,treatment,outcome
0,0.125730,0,0,0,-0.132105
1,0.640423,1,0,0,0.104900
30,0.345584,0,1,0,-0.200820
31,0.330437,1,1,0,-1.303157
60,0.189053,0,2,1,-0.522748
...,...,...,...,...,...
2911,-0.425905,1,97,0,-0.985005
2940,-1.087591,0,98,0,-1.244621
2941,0.560527,1,98,0,NaN
2970,0.082494,0,99,1,-0.464418


In [20]:
print(next_dt.isna().value_counts())
next_filled_dt = context_fill(next_dt, context_cols, N, m)
print(next_filled_dt.isna().value_counts())

c      t      patient_id  treatment  outcome
False  False  False       False      False      188
                                     True        12
Name: count, dtype: int64
c      t      patient_id  treatment  outcome
False  False  False       False      False      200
Name: count, dtype: int64


c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
